# Summary

1. energy_per_atom을 target property로 하여 약 27000개의 material project 데이터로 학습
2. SrTiO3의 Ti 원자의 위치를 변경해가며 이에 따른 energy_per_atom 변화곡선을 그림
3. 예전에는 DFT 결과와 모양은 비슷하게 나왔는데, 다시하니까 안된다;;



# Google Drive Mount

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
cd drive/My Drive

/content/drive/My Drive


In [3]:
ls

'읽을 예정인 논문 모음'/  '01. Backup Files'/   cgcnn/      GraphGAN/
'00. Github'/             '02. 데이터'/         CGCNN-HD/


# CGCNN github clone

In [4]:
#!git clone https://github.com/txie-93/cgcnn

# Pymatgen Install

In [5]:
!pip install pymatgen==2020.11.11

  Created wheel for pymatgen: filename=pymatgen-2020.11.11-cp37-cp37m-linux_x86_64.whl size=3573487 sha256=0381b3fc4a2f6d536cd5af1094f4d4fce8d8677f8921e993ad294384ee242413
  Stored in directory: /root/.cache/pip/wheels/b1/78/1c/a0dc6caae3d0bcae2f5db36569448d8f064b1cc0e900f0e7ca
Successfully built pymatgen
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
  Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [6]:
cd /content/drive/My Drive/cgcnn/data/material-data 


/content/drive/My Drive/cgcnn/data/material-data


In [7]:
ls

mp_data-27430.pickle  mp-ids-27430.csv  mp-ids-46744.csv
mp_data-46744.pickle  mp-ids-3402.csv   README.md


# material project id collection

In [8]:
import pandas as pd
import numpy as np

cgcnn 저자가 제공하는 46744개의 materials project id를 사용한다.

In [9]:
material_project_ids = pd.read_csv('mp-ids-46744.csv', header = None)
material_project_ids = np.array(material_project_ids).reshape(-1)
print(material_project_ids)
print(len(material_project_ids))

['mp-754118' 'mp-978908' 'mp-633688' ... 'mp-755142' 'mp-648469' 'mp-2327']
46744


In [10]:
material_project_ids[2]

'mp-633688'

# Custom Dataset Generation

In [11]:
from pymatgen.ext.matproj import MPRester
from tqdm import tqdm
import pickle

27430개의 materials project ids.csv 파일에서 energy_per_atom과 structure 정보를 읽어 data에 저장한다.

In [12]:
'''
data = {}
my_API_key = "pyvjZNMMGjtZskXhTQb"

with MPRester(my_API_key) as m:

    for mp_id in tqdm(material_project_ids):
        # target propery : energy_per_atom
        # number of data : about 20000
        data[mp_id] = (m.query(criteria = {'material_id': mp_id},
                               properties = ['energy_per_atom', 'structure']))
'''

'\ndata = {}\nmy_API_key = "pyvjZNMMGjtZskXhTQb"\n\nwith MPRester(my_API_key) as m:\n\n    for mp_id in tqdm(material_project_ids):\n        # target propery : energy_per_atom\n        # number of data : about 20000\n        data[mp_id] = (m.query(criteria = {\'material_id\': mp_id},\n                               properties = [\'energy_per_atom\', \'structure\']))\n'

매번 target propery를 읽어오는 것은 시간이 오래 걸리므로 pickle로 저장해둔다.

In [13]:
# save data
#with open('energy_per_atom_27430.pickle','wb') as fw:
#    pickle.dump(data, fw)

데이터 재사용시에는 load data 사용

In [14]:
# load data
with open('mp_data-46744.pickle', 'rb') as fr:
    data = pickle.load(fr)

In [15]:
data['mp-1000']

[{'energy_per_atom': -4.323689235, 'structure': Structure Summary
  Lattice
      abc : 5.013215958771575 5.013215958771575 5.013215958771575
   angles : 60.00000000000001 60.00000000000001 60.00000000000001
   volume : 89.09108390126052
        A : 0.0 3.544879 3.544879
        B : 3.544879 0.0 3.544879
        C : 3.544879 3.544879 0.0
  PeriodicSite: Ba (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
  PeriodicSite: Te (3.5449, 3.5449, 3.5449) [0.5000, 0.5000, 0.5000]}]

Custom Dataset으로 CGCNN 코드를 작동시킬 경우, 실질적으로 2개의 데이터를 새로 생성해야 한다. 

1. new_id_prop_data
2. new_crystal

In [16]:
# 1. new_id_prop_data 생성
new_id_prop_data = []

for key, value in data.items():

    if value == []:
        pass
    else:
        new_id_prop_data.append([key, value[0]['energy_per_atom']])

In [17]:
print('실제 유의미한 데이터의 수 : {}'.format(len(new_id_prop_data)))

실제 유의미한 데이터의 수 : 36832


In [18]:
# 2. new_crystal 생성
idx = 2
cif_id, target = new_id_prop_data[idx]
print('cif_id: {}'.format(cif_id))
print('target: {}'.format(target))

cif_id: mp-3799
target: -9.681103753333334


In [19]:
new_crystal_structure = data[cif_id][0]['structure']
new_crystal_structure

Structure Summary
Lattice
    abc : 3.849352 3.849352 6.919488
 angles : 90.0 90.0 90.0
 volume : 102.5295883081959
      A : 3.849352 0.0 0.0
      B : 0.0 3.849352 0.0
      C : 0.0 0.0 6.919488
PeriodicSite: Gd (0.0000, 1.9247, 5.3246) [0.0000, 0.5000, 0.7695]
PeriodicSite: Gd (1.9247, 0.0000, 1.5948) [0.5000, 0.0000, 0.2305]
PeriodicSite: S (1.9247, 0.0000, 4.4577) [0.5000, 0.0000, 0.6442]
PeriodicSite: S (0.0000, 1.9247, 2.4618) [0.0000, 0.5000, 0.3558]
PeriodicSite: F (1.9247, 1.9247, 0.0000) [0.5000, 0.5000, 0.0000]
PeriodicSite: F (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]

## cgcnn/cgcnn/data.py

In [20]:
from __future__ import print_function, division

import csv
import functools
import json
import os
import random
import warnings

import numpy as np
import torch
from pymatgen.core.structure import Structure
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataloader import default_collate
from torch.utils.data.sampler import SubsetRandomSampler

### get_train_val_test_loader

In [21]:
def get_train_val_test_loader(dataset, collate_fn = default_collate,
                              batch_size = 64, train_ratio = None,
                              val_ratio = 0.1, test_ratio = 0.1, return_test = False,
                              num_workers = 1, pin_memory = False, **kwargs):
    """
    Utility function for dividing a dataset to train, val, test datasets.
    !!! The dataset needs to be shuffled before using the function !!!

    Parameters
    ----------
    dataset: torch.utils.data.Dataset
             The full dataset to be divided.
    collate_fn: torch.utils.data.DataLoader
    batch_size: int
    train_ratio: float
    val_ratio: float
    test_ratio: float
    return_test: bool
        Whether to return the test dataset loader. If False, the last test_size 
        data will be hidden.
    num_workers: int
    pin_memory: bool

    Returns
    -------
    train_loader: torch.utils.data.DataLoader
        DataLoader that random samples the training data.
    val_loader: torch.utils.data.DataLoader
        DataLoader that random samples the validation data.
    (test_loader): torch.utils.data.DataLoader
        DataLoader that random samples the test data, returns if
        return_test = True.
    """
    total_size = len(dataset)

    if train_ratio is None:
        assert val_ratio + test_ratio < 1
        train_ratio = 1 - val_ratio - test_ratio
        print('[Warning] train_ratio is None, using all training data.')
    else:
        assert train_ratio + val_ratio + test_ratio <= 1
    
    indices = list(range(total_size))

    if kwargs['train_size']:
        train_size = kwargs['train_size']
    else:
        train_size = int(train_ratio * total_size)

    if kwargs['test_size']:
        test_size = kwargs['test_size']
    else:
        test_size = int(test_ratio * total_size)
    
    if kwargs['val_size']:
        valid_size = kwargs['val_size']
    else:
        valid_size = int(val_ratio * total_size)

    train_sampler = SubsetRandomSampler(indices[:train_size])
    val_sampler = SubsetRandomSampler(
        indices[-(valid_size + test_size):-test_size])
    
    if return_test:
        test_sampler = SubsetRandomSampler(indices[-test_size:])

    train_loader = DataLoader(dataset, batch_size = batch_size,
                              sampler = train_sampler,
                              num_workers = num_workers,
                              collate_fn = collate_fn, pin_memory = pin_memory)
    val_loader = DataLoader(dataset, batch_size = batch_size,
                            sampler = val_sampler,
                            num_workers = num_workers,
                            collate_fn = collate_fn, pin_memory = pin_memory)
    if return_test:
        test_loader = DataLoader(dataset, batch_size = batch_size,
                                 sampler = test_sampler,
                                 num_workers = num_workers,
                                 collate_fn = collate_fn, pin_memory = pin_memory)
    if return_test:
        return train_loader, val_loader, test_loader
    else:
        return train_loader, val_loader 
        

### collate_pool

In [22]:
def collate_pool(dataset_list):
    """
    Collate a list of data and return a batch for predicting crystal properties.

    Parameters
    ----------
    dataset_list: list of tuples for each data point.
        (atom_fea, nbr_fea, nbr_fea_idx, target)

        atom_fea: torch.Tensor shape (n_i, atom_fea_len)
        nbr_fea: torch.Tensor shape (n_i, M, nbr_fea_len)
        nbr_fea_idx: torch.LongTensor shape (n_i, M)
        target: torch.Tensor shape (1, )
        cif_id: str or int

    Returns
    -------
    N = sum(n_i); N0 = sum(i)

    batch_atom_fea: torch.Tensor shape (N, orig_atom_fea_len)
        Atom features from atom type
    batch_nbr_fea: torch.Tensor shape (N, M, nbr_fea_len)
        Bond features of each atom's M neighbors
    batch_nbr_fea_idx: torch.LongTensor shape (N, M)
        Indices of M neighbors of each atom
    crystal_atom_idx: list of torch.LongTensor of length N0
        Mapping from the crystal idx to atom idx
    target: torch.Tensor shape (N, 1)
        Target value for prediction
    batch_cif_ids: list
    """
    batch_atom_fea, batch_nbr_fea, batch_nbr_fea_idx = [], [], []
    crystal_atom_idx, batch_target = [], []
    batch_cif_ids = []
    base_idx = 0

    for i, ((atom_fea, nbr_fea, nbr_fea_idx), target, cif_id) in enumerate(dataset_list):

        n_i = atom_fea.shape[0]  # number of atoms for this crystal
        
        batch_atom_fea.append(atom_fea)
        batch_nbr_fea.append(nbr_fea)
        batch_nbr_fea_idx.append(nbr_fea_idx + base_idx)

        new_idx = torch.LongTensor(np.arange(n_i) + base_idx)

        crystal_atom_idx.append(new_idx)
        batch_target.append(target)
        batch_cif_ids.append(cif_id)

        base_idx += n_i

    return (torch.cat(batch_atom_fea, dim = 0),
            torch.cat(batch_nbr_fea, dim = 0),
            torch.cat(batch_nbr_fea_idx, dim = 0),
            crystal_atom_idx), \
            torch.stack(batch_target, dim = 0), \
            batch_cif_ids


### GaussianDistance

In [23]:
class GaussianDistance(object):
    """
    Expands the distance by Gaussian basis.

    Unit: angstrom
    """
    def __init__(self, dmin, dmax, step, var = None):
        """
        Parameters
        ----------
        dmin: float
            Minimum interatomic distance
        dmax: float
            Maximum interatomic distance
        step: float
            Step size for the Gaussian filter
        """
        assert dmin < dmax
        assert dmax - dmin > step
        self.filter = np.arange(dmin, dmax + step, step)
        if var is None:
            var = step
        self.var = var

    def expand(self, distances):
        """
        Apply Gaussian distances filter to a numpy distance array

        Parameters
        ----------
        distance: np.array shape n-d array
            A distance matrix of any shape
        
        Returns
        -------
        expanded_distance: shape (n+1)-d array
            Expanded distance matrix with the last dimension of length
            len(self.filter)
        """
        return np.exp(-(distances[..., np.newaxis] - self.filter)**2 / self.var**2)
        

### AtomInitializer

In [24]:
class AtomInitializer(object):
    """
    Base class for initializing the vector representation for atoms.
    !!! Use one AtomInitializer per dataset !!!
    """
    def __init__(self, atom_types):
        self.atom_types = set(atom_types)
        self._embedding = {}

    def get_atom_fea(self, atom_type):
        assert atom_type in self.atom_types
        return self._embedding[atom_type]

    def load_state_dict(self, state_dict):
        self._embedding = state_dict
        self.atom_types = set(self._embedding.keys())
        self._decodedict = {idx: atom_type for atom_type, idx in 
                            self._embedding.items()}

    def state_dict(self):
        return self._embedding

    def decode(self, idx):
        if not hasattr(self, '_decodedict'):
            self._decodedict = {idx: atom_type for atom_type, idx in 
                                self._embedding.items()}
        return self._decodedict[idx]
        

### AtomCustomJSONInitializer

In [25]:
class AtomCustomJSONInitializer(AtomInitializer):
    """
    Initialize atom feature vectors using a JSON file, which is a python
    dictionary mapping from element number to a list representing the 
    feature vector of the element.

    Parameters
    ----------
    elem_embedding_file: str
        The path to the .json file
    """
    def __init__(self, elem_embedding_file):
        with open(elem_embedding_file) as f:
            elem_embedding = json.load(f)
        elem_embedding = {int(key): value for key, value
                          in elem_embedding.items()}
        atom_types = set(elem_embedding.keys())
        super(AtomCustomJSONInitializer, self).__init__(atom_types)
        for key, value in elem_embedding.items():
            self._embedding[key] = np.array(value, dtype = float)
            

### CIFData

In [26]:
class CIFData(Dataset):
    """
    The CIFData dataset is a wrapper for a dataset where the crystal structures
    are stored in the form of CIF files. The dataset should have the following 
    directory structure:

    root_dir
    ├── id_prop.csv
    ├── atom_init.json
    ├── id0.cif
    ├── id1.cif
    ├── ...

    id_prop.csv: a CSV file with two columns. The first column recodes a 
    unique ID for each crystal, and the second column recodes the value of
    target property.

    atom_init.json: a JSON file that stores the initialization vector for each
    element.

    ID.cif: a CIF file that recodes the crystal structure, where ID is the
    unique ID for the crystal.

    Parameters
    ----------
    root_dir: str
        The path to the root directory of the dataset
    max_num_nbr: int
        The maximum number of neighbors while constructing the crystal graph
    radius: float
        The cutoff radius for searching neighbors
    dmin: float
        The minimum distance for constructing GaussianDistance
    step: float
        The step size for constructing GaussianDistance
    random_seed: int
        Random seed for shuffling the dataset

    Returns
    -------
    atom_fea: torch.Tensor shape (n_i, atom_fea_len)
    nbr_fea: torch.Tensor shape (n_i, M, nbr_fea_len)
    nbr_fea_idx: torch.LongTensor shape (n_i, M)
    target: torch.Tensor shape (1, )
    cif_id: str or int
    """
    def __init__(self, root_dir, max_num_nbr = 12, radius = 8, dmin = 0, step = 0.2,
                 random_seed = 123):
        self.root_dir = root_dir
        self.max_num_nbr, self.radius = max_num_nbr, radius

        assert os.path.exists(root_dir), 'root_dir does not exist!'
        id_prop_file = os.path.join(self.root_dir, 'id_prop.csv')
        assert os.path.exists(id_prop_file), 'id_prop.csv does not exist!'

        with open(id_prop_file) as f:
            reader = csv.reader(f)
        #    self.id_prop_data = [row for row in reader]
        # ==================================
        self.id_prop_data = new_id_prop_data # Custom dataset 입력
        # ==================================
        
        random.seed(random_seed)
        random.shuffle(self.id_prop_data)

        atom_init_file = os.path.join(self.root_dir, 'atom_init.json')
        assert os.path.exists(atom_init_file), 'atom_init.json does not exist!'
        
        self.ari = AtomCustomJSONInitializer(atom_init_file)
        self.gdf = GaussianDistance(dmin = dmin, dmax = self.radius, step = step)

    def __len__(self):
        return len(self.id_prop_data)

    @functools.lru_cache(maxsize = None)  # Cache loaded structures
    def __getitem__(self, idx):
        cif_id, target = self.id_prop_data[idx]
        #crystal = Structure.from_file(os.path.join(self.root_dir, cif_id + '.cif'))
        
        # ====================================
        crystal = data[cif_id][0]['structure'] # Custom dataset 입력
        # ====================================
        
        atom_fea = np.vstack([self.ari.get_atom_fea(crystal[i].specie.number)
                              for i in range(len(crystal))])
        atom_fea = torch.Tensor(atom_fea)

        all_nbrs = crystal.get_all_neighbors(self.radius, include_index = True)
        all_nbrs = [sorted(nbrs, key = lambda x: x[1]) for nbrs in all_nbrs]

        nbr_fea_idx, nbr_fea = [], []
        for nbr in all_nbrs:
            if len(nbr) < self.max_num_nbr:
                warnings.warn('{} not find enough neighbors to build graph. '
                              'If it happens frequently, consider increase '
                              'radius.'.format(cif_id))
                nbr_fea_idx.append(list(map(lambda x: x[2], nbr)) +
                                   [0] * (self.max_num_nbr - len(nbr)))
                nbr_fea.append(list(map(lambda x: x[1], nbr)) + 
                               [self.radius + 1.] * (self.max_num_nbr - len(nbr)))
            else:
                nbr_fea_idx.append(list(map(lambda x: x[2],
                                            nbr[:self.max_num_nbr])))
                nbr_fea.append(list(map(lambda x: x[1], 
                                        nbr[:self.max_num_nbr])))
                
        nbr_fea_idx, nbr_fea = np.array(nbr_fea_idx), np.array(nbr_fea)
        nbr_fea = self.gdf.expand(nbr_fea)
        atom_fea = torch.Tensor(atom_fea)
        nbr_fea = torch.Tensor(nbr_fea)
        nbr_fea_idx = torch.LongTensor(nbr_fea_idx)
        target = torch.Tensor([float(target)])

        return (atom_fea, nbr_fea, nbr_fea_idx), target, cif_id


## cgcnn/cgcnn/model.py

In [27]:
from __future__ import print_function, division

import torch
import torch.nn as nn

### ConvLayer

In [28]:
class ConvLayer(nn.Module):
    """
    Convolutional operation on graphs
    """
    def __init__(self, atom_fea_len, nbr_fea_len):
        """
        Initialize ConvLayer.

        Parameters
        ----------
        atom_fea_len: int
            Number of atom hidden features.
        nbr_fea_len: int
            Number of bond features.
        """
        super(ConvLayer, self).__init__()
        self.atom_fea_len = atom_fea_len
        self.nbr_fea_len = nbr_fea_len
        self.fc_full = nn.Linear(2 * self.atom_fea_len + self.nbr_fea_len,
                                 2 * self.atom_fea_len)
        self.sigmoid = nn.Sigmoid()
        self.softplus1 = nn.Softplus()
        self.bn1 = nn.BatchNorm1d(2 * self.atom_fea_len)
        self.bn2 = nn.BatchNorm1d(self.atom_fea_len)
        self.softplus2 = nn.Softplus()

    def forward(self, atom_in_fea, nbr_fea, nbr_fea_idx):
        """
        Forward pass

        N: Total number of atoms in the batch
        M: Max number of neighbors

        Parameters
        ----------
        atom_in_fea: Variable(torch.Tensor) shape (N, atom_fea_len)
            Atom hidden features before convolution
        nbr_fea: Variable(torch.Tensor) shape (N, M, nbr_fea_len)
            Bond features of each atom's M neighbors
        nbr_fea_idx: torch.LongTensor shape (N, M)
            Indices of M neighbors of each atom
        
        Returns
        -------
        atom_out_fea: nn.Variable shape (N, atom_fea_len)
            Atom hidden features after convolution
        """
        # TODO will there be problems with the index zero padding?
        N, M = nbr_fea_idx.shape

        # convolution
        atom_nbr_fea = atom_in_fea[nbr_fea_idx, :]
        total_nbr_fea = torch.cat(
            [atom_in_fea.unsqueeze(1).expand(N, M, self.atom_fea_len),
             atom_nbr_fea, nbr_fea], dim = 2)
        
        total_gated_fea = self.fc_full(total_nbr_fea)
        total_gated_fea = self.bn1(total_gated_fea.view(
            -1, self.atom_fea_len * 2)).view(N, M, self.atom_fea_len * 2)
        
        nbr_filter, nbr_core = total_gated_fea.chunk(2, dim = 2)
        nbr_filter = self.sigmoid(nbr_filter)
        nbr_core = self.softplus1(nbr_core)
        nbr_sumed = torch.sum(nbr_filter * nbr_core, dim = 1)
        nbr_sumed = self.bn2(nbr_sumed)
        out = self.softplus2(atom_in_fea + nbr_sumed)

        return out

### CrystalGraphConvNet

In [29]:
class CrystalGraphConvNet(nn.Module):
    """
    Create a crystal graph convolutional neural network for predicting total
    material properties.
    """
    def __init__(self, orig_atom_fea_len, nbr_fea_len,
                 atom_fea_len = 64, n_conv = 3, h_fea_len = 128, n_h = 1,
                 classification = False):
        """
        Initialize CrystalGraphConvNet.

        Parameters
        ----------
        orig_atom_fea_len: int
            Number of atom features in the input.
        nbr_fea_len: int
            Number of bond features.
        atom_fea_len: int
            Number of hidden atom features in the convolutional layers
        n_conv: int
            Number of convolutional layers
        h_fea_len: int
            Number of hidden features after pooling
        n_h: int
            Number of hidden layers after pooling
        """
        super(CrystalGraphConvNet, self).__init__()
        self.classification = classification 
        self.embedding = nn.Linear(orig_atom_fea_len, atom_fea_len)
        self.convs = nn.ModuleList([ConvLayer(atom_fea_len = atom_fea_len,
                                              nbr_fea_len = nbr_fea_len)
                                              for _ in range(n_conv)])
        self.conv_to_fc = nn.Linear(atom_fea_len, h_fea_len)
        self.conv_to_fc_softplus = nn.Softplus()

        if n_h > 1:
            self.fcs = nn.ModuleList([nn.Linear(h_fea_len, h_fea_len)
                                      for _ in range(n_h - 1)])
            self.softpluses = nn.ModuleList([nn.Softplus()
                                             for _ in range(n_h - 1)])
        
        if self.classification:
            self.fc_out = nn.Linear(h_fea_len, 2)
        else:
            self.fc_out = nn.Linear(h_fea_len, 1)
        
        if self.classification:
            self.logsoftmax = nn.LogSoftmax(dim = 1)
            self.dropout = nn.Dropout()

    def forward(self, atom_fea, nbr_fea, nbr_fea_idx, crystal_atom_idx):
        """
        Forward pass

        N: Total number of atoms in the batch
        M: Max number of neighbors
        N0: Total number of crystals in the batch

        Parameters
        ----------
        atom_fea: Variable(torch.Tensor) shape (N, orig_atom_fea_len)
            Atom features from atom type
        nbr_fea: Variable(torch.Tensor) shape (N, M, nbr_fea_len)
            Bond features of each atom's M neighbors
        nbr_fea_idx: torch.LongTensor shape (N, M)
            Indices of M neighbors of each atom
        crystal_atom_idx: list of torch.LongTensor of length N0
            Mapping from the crystal idx to atom idx

        Returns
        -------
        prediction: nn.Variable shape (N, )
            Atom hidden features after convolution
        """
        atom_fea = self.embedding(atom_fea)
        for conv_func in self.convs:
            atom_fea = conv_func(atom_fea, nbr_fea, nbr_fea_idx)
        
        crys_fea = self.pooling(atom_fea, crystal_atom_idx)
        crys_fea = self.conv_to_fc(self.conv_to_fc_softplus(crys_fea))
        crys_fea = self.conv_to_fc_softplus(crys_fea)

        if self.classification:
            crys_fea = self.dropout(crys_fea)
        
        if hasattr(self, 'fcs') and hasattr(self, 'softpluses'):
            for fc, softplus in zip(self.fcs, self.softpluses):
                crys_fea = softplus(fc(crys_fea))
        
        out = self.fc_out(crys_fea)

        if self.classification:
            out = self.logsoftmax(out)
        
        return out

    def pooling(self, atom_fea, crystal_atom_idx):
        """
        Pooling the atom features to crystal features

        N: Total number of atoms in the batch
        N0: Total number of crystals in the batch

        Parameters
        ----------
        atom_fea: Variable(torch.Tensor) shape (N, atom_fea_len)
            Atom feature vectors of the batch
        crystal_atom_idx: list of torch.LongTensor of length N0
            Mapping from the crystal idx to atom idx
        """
        assert sum([len(idx_map) for idx_map in crystal_atom_idx]) == \
            atom_fea.data.shape[0]
        summed_fea = [torch.mean(atom_fea[idx_map], dim = 0, keepdim = True)
                      for idx_map in crystal_atom_idx]
        return torch.cat(summed_fea, dim = 0)


## cgcnn/main.py

In [30]:
import argparse
import os
import shutil
import sys
import time
import warnings
from random import sample

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import metrics
from torch.autograd import Variable
from torch.optim.lr_scheduler import MultiStepLR

#from cgcnn.data import CIFData
#from cgcnn.data import collate_pool, get_train_val_test_loader
#from cgcnn.model import CrystalGraphConvNet

### Args

In [31]:
class Args:
    def __init__(self):

        self.data_options = './data/sample-regression'
        self.task = 'regression' 
        self.disable_cuda = False # help = Disable CUDA, action = 'store_ture'
        self.workers = 0
        self.epochs = 30
        self.start_epoch = 0
        self.batch_size = 256
        self.lr = 0.01
        self.lr_milestones = [100]
        self.momentum = 0.9
        self.weight_decay = 0
        self.print_freq = 10
        self.resume = None # default = '', help = path to latest checkpoint(default:None)

        # train_group
        self.train_ratio = None 
        self.train_size = None

        # valid_group
        self.val_ratio = 0.1 
        self.val_size = 1000 

        # test_group : add_mutually_exclusive_group
        self.test_ratio = 0.1 
        self.test_size = 1000 
        self.optim = 'Adam'
        self.atom_fea_len = 64 
        self.h_fea_len = 128
        self.n_conv = 3
        self.n_h = 1
args=Args()
args

In [32]:
args.cuda = not args.disable_cuda and torch.cuda.is_available()
print(args.cuda) # True if GPU on

True


In [33]:
if args.task == 'regression':
    best_mae_error = 1e10
else:
    best_mae_error = 0.0

print(best_mae_error)

10000000000.0


### Normalizer

In [34]:
class Normalizer(object):
    """ Normalize a Tensor and restore it later. """

    def __init__(self, tensor):
        """ tensor is taken as a sample to calculate the mean and std """
        self.mean = torch.mean(tensor)
        self.std = torch.std(tensor)

    def norm(self, tensor):
        return (tensor - self.mean) / self.std
    
    def denorm(self, normed_tensor):
        return normed_tensor * self.std + self.mean

    def state_dict(self):
        return {'mean': self.mean, 
                'std': self.std}
    
    def load_state_dict(self, state_dict):
        self.mean = state_dict['mean']
        self.std = state_dict['std']
        

### mae

In [35]:
def mae(prediction, target):
    """
    Computes the mean absolute error between prediction and target

    Parameters
    ----------
    prediction: torch.Tensor (N, 1)
    target: torch.Tensor (N, 1)
    """
    return torch.mean(torch.abs(target - prediction))

### class_eval

In [36]:
def class_eval(prediction, target):

    prediction = np.exp(prediction.numpy())
    target = target.numpy()

    pred_label = np.argmax(prediction, axis = 1)
    target_label = np.squeeze(target)

    if not target_label.shape:
        target_label = np.asarray([target_label])
    
    if prediction.shape[1] == 2:
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(
            target_label, pred_label, average = 'binary')
        auc_score = metrics.roc_auc_score(target_label, prediction[:, 1])
        accuracy = metrics.accuracy_score(target_label, pred_label)
    else:
        raise NotImplementedError
    return accuracy, precision, recall, fscore, auc_score

### AverageMeter

In [37]:
class AverageMeter(object):
    """ Computes and stores the average and current value """
    
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n = 1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

### save_checkpoint

In [38]:
def save_checkpoint(state, is_best, filename="checkpoint_46744_energy_per_atom.pth.tar"):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best_46744_energy_per_atom.pth.tar')

### adjust_learning_rate

In [39]:
def adjust_learning_rate(optimizer, epoch, k):
    """ Sets the learning rate to the initial LR decayed by 10 every k epochs """
    assert type(k) is int
    lr = args.lr * (0.1 ** (epoch // k))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
        

### train

In [40]:
def train(train_loader, model, criterion, optimizer, epoch, normalizer):

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()

    if args.task == 'regression':
        mae_errors = AverageMeter()
    else:
        accuracies = AverageMeter()
        precisions = AverageMeter()
        recalls = AverageMeter()
        fscores = AverageMeter()
        auc_scores = AverageMeter()
    
    # switch to train mode
    model.train()

    end = time.time()

    for i, (input, target, _) in enumerate(train_loader):

        # measure data loading time
        data_time.update(time.time() - end)

        if args.cuda:
            input_var = (Variable(input[0].cuda(non_blocking = True)),
                         Variable(input[1].cuda(non_blocking = True)),
                         input[2].cuda(non_blocking = True),
                         [crys_idx.cuda(non_blocking = True) for crys_idx in input[3]])
        else:
            input_var = (Variable(input[0]),
                         Variable(input[1]),
                         input[2],
                         input[3])
        
        # normalize target
        if args.task == 'regression':
            target_normed = normalizer.norm(target)
        else:
            target_normed = target.view(-1).long()
        
        if args.cuda:
            target_var = Variable(target_normed.cuda(non_blocking = True))
        else:
            target_var = Variable(target_normed)

        # compute output
        output = model(*input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        if args.task == 'regression':
            mae_error = mae(normalizer.denorm(output.data.cpu()), target)
            losses.update(loss.data.cpu(), target.size(0))
            mae_errors.update(mae_error, target.size(0))
        else:
            accuracy, precision, recall, fscore, auc_score = \
                class_eval(output.data.cpu(), target)
            losses.update(loss.data.cpu().item(), target.size(0))
            accuracies.update(accuracy, target.size(0))
            precisions.update(precision, target.size(0))
            recalls.update(recall, target.size(0))
            fscores.update(fscore, target.size(0))
            auc_scores.update(auc_score, target.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            if args.task == 'regression':
                print('Epoch: [{0}][{1}/{2}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'MAE {mae_errors.val:.3f} ({mae_errors.avg:.3f})'.format(
                          epoch, i, len(train_loader), batch_time = batch_time,
                          data_time = data_time, loss = losses, mae_errors = mae_errors))
            else:
                print('Epoch: [{0}][{1}/{2}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Accu {accu.val:.3f} ({accu.avg:.3f})\t'
                      'Precision {prec.val:.3f} ({prec.avg:.3f})\t'
                      'Recall {recall.val:.3f} ({recall.avg:.3f})\t'
                      'F1 {f1.val:.3f} ({f1.avg:.3f})\t'
                      'AUC {auc.val:.3f} ({auc.avg:.3f})'.format(
                          epoch, i, len(train_loader), batch_time = batch_time,
                          data_time = data_time, loss = losses, accu = accuracies,
                          prec = precisions, recall = recalls, f1 = fscores,
                          auc = auc_scores))


### validate

In [41]:
def validate(val_loader, model, criterion, normalizer, test = False):

    batch_time = AverageMeter()
    losses = AverageMeter()

    if args.task == 'regression':
        mae_errors = AverageMeter()
    else:
        accuracies = AverageMeter()
        precisions = AverageMeter()
        recalls = AverageMeter()
        fscores = AverageMeter()
        auc_scores = AverageMeter()
    
    if test:
        test_targets = []
        test_preds = []
        test_cif_ids = []

    # switch to evaluate mode
    model.eval()

    end = time.time()

    for i, (input, target, batch_cif_ids) in enumerate(val_loader):

        if args.cuda:
            with torch.no_grad():
                input_var = (Variable(input[0].cuda(non_blocking = True)),
                             Variable(input[1].cuda(non_blocking = True)),
                             input[2].cuda(non_blocking = True),
                             [crys_idx.cuda(non_blocking = True) for crys_idx in input[3]])
        else:
            with torch.no_grad():
                input_var = (Variable(input[0]),
                             Variable(input[1]),
                             input[2],
                             input[3])
        if args.task == 'regression':
            target_normed = normalizer.norm(target)
        else:
            target_normed = target.view(-1).long()

        if args.cuda:
            with torch.no_grad():
                target_var = Variable(target_normed.cuda(non_blocking = True))
        else:
            with torch.no_grad():
                target_var = Variable(target_normed)
        
        # compute output
        output = model(*input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        if args.task == 'regression':
            mae_error = mae(normalizer.denorm(output.data.cpu()), target)
            losses.update(loss.data.cpu().item(), target.size(0))
            mae_errors.update(mae_error, target.size(0))

            if test:
                test_pred = normalizer.denorm(output.data.cpu())
                test_target = target
                test_preds += test_pred.view(-1).tolist()
                test_targets += test_target.view(-1).tolist()
                test_cif_ids += batch_cif_ids
        else:
            accuracy, precision, recall, fscore, auc_score = \
                class_eval(output.data.cpu(), target)
            
            losses.update(loss.data.cpu().item(), target.size(0))
            accuracies.update(accuracy, target.size(0))
            precisions.update(precision, target.size(0))
            recalls.update(recall, target.size(0))
            fscores.update(fscore, target.size(0))
            auc_scores.update(auc_score, target.size(0))

            if test:
                test_pred = torch.exp(output.data.cpu())
                test_target = target
                assert test_pred.shape[1] == 2
                test_preds += test_pred[:, 1].tolist()
                test_targets += test_target.view(-1).tolist()
                test_cif_ids += batch_cif_ids
        
        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args.print_freq == 0:
            if args.task == 'regression':
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'MAE {mae_errors.val:.3f} ({mae_errors.avg:.3f})'.format(
                          i, len(val_loader), batch_time = batch_time, loss = losses,
                          mae_errors = mae_errors))
            else:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Accu {accu.val:.3f} ({accu.avg:.3f})\t'
                      'Precision {prec.val:.3f} ({prec.avg:.3f})\t'
                      'Recall {recall.val:.3f} ({recall.avg:.3f})\t'
                      'F1 {f1.val:.3f} ({f1.avg:.3f})\t'
                      'AUC {auc.val:.3f} ({auc.avg:.3f})'.format(
                          i, len(val_loader), batch_time = batch_time, loss = losses,
                          accu = accuracies, prec = precisions, recall = recalls,
                          f1 = fscores, auc = auc_scores))                
    if test:
        star_label = '**'
        import csv
        with open('test_results.csv', 'w') as f:
            writer = csv.writer(f)

            for cif_id, target, pred in zip(test_cif_ids, test_targets, test_preds):
                writer.writerow((cif_id, target, pred))
    else:
        star_label = '*'


    if args.task == 'regression':
        print(' {star} MAE {mae_errors.avg:.3f}'.format(star = star_label,
                                                        mae_errors = mae_errors))
        return mae_errors.avg
    else:
        print(' {star} AUC {auc.avg:.3f}'.format(star = star_label,
                                                 auc = auc_scores))
        return auc_scores.avg
        

### main()

In [42]:
global args, best_mae_error
print(args)
print(best_mae_error)

10000000000.0


In [43]:
cd /content/drive/My Drive/cgcnn/ 

/content/drive/My Drive/cgcnn


In [44]:
ls

cgcnn/
checkpoint2.pth.tar
checkpoint_46744_energy_per_atom.pth.tar
checkpoint.pth.tar
data/
LICENSE
main.py
model_best2.pth.tar
model_best_46744_energy_per_atom.pth.tar
model_best.pth.tar
predict.py
pre-trained/
README.md
SrTiO3_folder/
test_results.csv


In [45]:
# load data
dataset = CIFData(args.data_options) # ./data/sample-regression
print(dataset)
print(len(dataset))

36832


In [46]:
collate_fn = collate_pool
print(collate_fn)

<function collate_pool at 0x7fd481ea1dd0>


데이터가 제한되어 있으므로 80%까지 훈련 데이터로 사용한다.

In [47]:
train_loader, val_loader, test_loader = get_train_val_test_loader(
                                            dataset = dataset,
                                            collate_fn = collate_fn,
                                            batch_size = args.batch_size,
                                            train_ratio = 0.8,
                                            num_workers = args.workers,
                                            val_ratio = 0.1,
                                            test_ratio = 0.1,
                                            pin_memory = args.cuda,
                                            train_size = args.train_size,
                                            val_size = args.val_size,
                                            test_size = args.test_size,
                                            return_test = True)
print(train_loader, len(train_loader))
print(val_loader, len(val_loader))
print(test_loader, len(test_loader))

<torch.utils.data.dataloader.DataLoader object at 0x7fd4f3915ad0> 116
<torch.utils.data.dataloader.DataLoader object at 0x7fd4f39159d0> 4
<torch.utils.data.dataloader.DataLoader object at 0x7fd4f3915690> 4


In [49]:
# obtain target value normalizer
if args.task == 'classification':
    normalizer = Normalizer(torch.zeros(2))
    normalizer.load_state_dict({'mean': 0., 'std': 1.})
else:
    if len(dataset) < 500:
        print('Dataset has less than 500 data points. '
              'Lower accuracy is expected. ')
        sample_data_list = [dataset[i] for i in range(len(dataset))]
    else:
        print('Dataset has more than 500 data points. good!')
        sample_data_list = [dataset[i] for i in sample(population = range(len(dataset)),
                                                       k = 500)]
print(sample_data_list)

Streaming output truncated to the last 5000 lines.
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        ...,

        [[1.2337e-30, 7.3288e-24, 5.8920e-18,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 4.0549e-40, 2.7924e-32,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.00

In [50]:
_, sample_target, _ = collate_pool(sample_data_list)
normalizer = Normalizer(sample_target)

print(sample_target)
print(normalizer)

tensor([[ -5.6705],
        [ -9.9795],
        [ -4.4060],
        [ -7.3827],
        [ -7.1923],
        [ -5.3040],
        [ -6.6125],
        [ -6.6916],
        [ -7.5281],
        [ -4.0837],
        [ -5.8688],
        [ -2.9623],
        [ -6.6207],
        [ -2.6590],
        [ -1.7817],
        [ -4.5522],
        [ -5.1878],
        [ -5.0179],
        [ -7.0544],
        [ -5.0042],
        [ -6.3545],
        [ -6.4692],
        [ -5.2336],
        [ -5.2028],
        [ -7.2813],
        [ -3.5575],
        [ -7.0674],
        [ -6.3539],
        [ -6.8249],
        [ -6.3195],
        [ -6.6333],
        [ -5.2152],
        [ -6.5106],
        [ -4.3064],
        [ -6.8314],
        [ -6.3651],
        [ -5.6760],
        [ -5.5664],
        [ -2.9228],
        [ -4.6327],
        [ -4.5979],
        [ -4.3157],
        [ -3.3343],
        [ -6.3898],
        [ -3.2964],
        [ -2.4266],
        [ -4.8155],
        [ -7.8595],
        [ -5.1939],
        [ -6.1982],


In [51]:
# build model
structures, _, _ = dataset[0] # (atom_fea, nbr_fea, nbr_fea_idx), target, cif_id
print(type(structures))
print(structures) # (atom_fea, nbr_fea, nbr_fea_idx)

<class 'tuple'>
(tensor([[0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0.,

In [52]:
orig_atom_fea_len = structures[0].shape[-1]
nbr_fea_len = structures[1].shape[-1]

print('orig_atom_fea_len: {}'.format(orig_atom_fea_len))
print('nbr_fea_len: {}'.format(nbr_fea_len))

orig_atom_fea_len: 92
nbr_fea_len: 41


In [53]:
model = CrystalGraphConvNet(orig_atom_fea_len = orig_atom_fea_len,
                            nbr_fea_len = nbr_fea_len,
                            atom_fea_len = args.atom_fea_len,
                            n_conv = args.n_conv,
                            h_fea_len = args.h_fea_len,
                            n_h = args.n_h,
                            classification = True if args.task == 'classification' 
                                                                       else False)
print(model)

CrystalGraphConvNet(
  (embedding): Linear(in_features=92, out_features=64, bias=True)
  (convs): ModuleList(
    (0): ConvLayer(
      (fc_full): Linear(in_features=169, out_features=128, bias=True)
      (sigmoid): Sigmoid()
      (softplus1): Softplus(beta=1, threshold=20)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (softplus2): Softplus(beta=1, threshold=20)
    )
    (1): ConvLayer(
      (fc_full): Linear(in_features=169, out_features=128, bias=True)
      (sigmoid): Sigmoid()
      (softplus1): Softplus(beta=1, threshold=20)
      (bn1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (softplus2): Softplus(beta=1, threshold=20)
    )
    (2): ConvLayer(
      (fc_full): Linear(in_features=169, out_featu

In [54]:
if args.cuda:
    model.cuda()

In [55]:
# define loss function and optimizer
if args.task == 'classification':
    criterion = nn.NLLLoss() # Negative Log Likelihood loss
else:
    criterion = nn.MSELoss() # Mean Squared Error Loss

if args.optim == 'SGD':
    optimizer = optim.SGD(params = model.parameters(), lr = args.lr,
                          momentum = args.momentum,
                          weight_decay = args.weight_decay)
elif args.optim == 'Adam':
    optimizer = optim.Adam(params = model.parameters(), lr = args.lr,
                           weight_decay = args.weight_decay)
else:
    raise NameError('Only SGD or Adam is allowed as optimizer')

print('criterion: {}'.format(criterion))
print('optimizer: {}'.format(optimizer))

criterion: MSELoss()
optimizer: Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.01
    weight_decay: 0
)


In [56]:
# optionally resume from a checkpoint
if args.resume:
    if os.path.isfile(args.resume):
        print("=> loading checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        args.start_epoch = checkpoint['epoch']
        best_mae_error = checkpoint['best_mae_error']

        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        normalizer.load_state_dict(checkpoint['normalizer'])

        print("=> loaded checkpoint '{}' (epoch {})".format(args.resume,
                                                             checkpoint['epoch']))
    else:
        print("=> No checkpoint found at '{}'".format(args.resume))

In [57]:
scheduler = MultiStepLR(optimizer = optimizer, milestones = args.lr_milestones,
                        gamma = 0.1)
print(scheduler)

이미 학습된 모델을 다시 사용할 경우는 아래 훈련 코드는 작동하지 말 것!

In [58]:
from tqdm import tqdm

for epoch in tqdm(range(args.start_epoch, args.epochs)):

    # train for one epoch
    train(train_loader = train_loader,
          model = model,
          criterion = criterion,
          optimizer = optimizer,
          epoch = epoch,
          normalizer = normalizer)
    
    # evaluate on validation set
    mae_error = validate(val_loader = val_loader,
                         model = model,
                         criterion = criterion,
                         normalizer = normalizer,
                         test = False)
    
    if mae_error != mae_error: # nan이면 비교 불가여서 False
        print('Exit due to NaN')
        sys.exit(1)

    scheduler.step()

    # remember the best mae_error and save checkpoint.
    if args.task == 'regression':
        is_best = mae_error < best_mae_error
        best_mae_error = min(mae_error, best_mae_error)
    else:
        is_best = mae_error > best_mae_error
        bese_mae_error = max(mae_error, best_mae_error)
    
    save_checkpoint(state = {'epoch': epoch + 1,
                             'state_dict': model.state_dict(),
                             'best_mae_error': best_mae_error,
                             'optimizer': optimizer.state_dict(),
                             'normalizer': normalizer.state_dict(),
                             'args': vars(args)}, is_best = is_best)
    

  0%|          | 0/30 [00:00<?, ?it/s]

Epoch: [0][0/116]	Time 11.620 (11.620)	Data 11.124 (11.124)	Loss 1.2148 (1.2148)	MAE 1.517 (1.517)
Epoch: [0][10/116]	Time 10.722 (10.671)	Data 10.377 (10.308)	Loss 0.8012 (4.0723)	MAE 1.243 (2.611)
Epoch: [0][20/116]	Time 10.095 (10.525)	Data 9.790 (10.179)	Loss 0.7380 (2.5535)	MAE 1.281 (2.004)
Epoch: [0][30/116]	Time 11.118 (10.528)	Data 10.798 (10.188)	Loss 0.7069 (1.9819)	MAE 1.215 (1.760)
Epoch: [0][40/116]	Time 11.249 (10.424)	Data 10.927 (10.087)	Loss 0.3658 (1.6240)	MAE 0.848 (1.576)
Epoch: [0][50/116]	Time 14.201 (10.572)	Data 13.772 (10.233)	Loss 0.2475 (1.3537)	MAE 0.616 (1.397)
Epoch: [0][60/116]	Time 11.584 (10.649)	Data 11.218 (10.308)	Loss 0.1980 (1.1669)	MAE 0.567 (1.265)
Epoch: [0][70/116]	Time 14.278 (10.669)	Data 13.968 (10.330)	Loss 0.1657 (1.0253)	MAE 0.527 (1.158)
Epoch: [0][80/116]	Time 10.937 (10.653)	Data 10.627 (10.315)	Loss 0.1035 (0.9141)	MAE 0.424 (1.072)
Epoch: [0][90/116]	Time 10.911 (10.686)	Data 10.573 (10.347)	Loss 0.0900 (0.8255)	MAE 0.373 (0.999)
Ep

  3%|▎         | 1/30 [21:08<10:13:00, 1268.29s/it]

Epoch: [1][0/116]	Time 0.462 (0.462)	Data 0.056 (0.056)	Loss 0.0727 (0.0727)	MAE 0.328 (0.328)
Epoch: [1][10/116]	Time 0.310 (0.343)	Data 0.021 (0.022)	Loss 0.0496 (0.0864)	MAE 0.288 (0.361)
Epoch: [1][20/116]	Time 0.325 (0.334)	Data 0.018 (0.020)	Loss 0.0844 (0.0841)	MAE 0.355 (0.355)
Epoch: [1][30/116]	Time 0.297 (0.330)	Data 0.017 (0.019)	Loss 0.0693 (0.0841)	MAE 0.316 (0.357)
Epoch: [1][40/116]	Time 0.306 (0.329)	Data 0.020 (0.019)	Loss 0.0701 (0.0859)	MAE 0.356 (0.359)
Epoch: [1][50/116]	Time 0.326 (0.329)	Data 0.018 (0.019)	Loss 0.0944 (0.0847)	MAE 0.393 (0.358)
Epoch: [1][60/116]	Time 0.334 (0.329)	Data 0.018 (0.019)	Loss 0.0625 (0.0840)	MAE 0.336 (0.354)
Epoch: [1][70/116]	Time 0.357 (0.329)	Data 0.021 (0.019)	Loss 0.0574 (0.0817)	MAE 0.297 (0.348)
Epoch: [1][80/116]	Time 0.315 (0.329)	Data 0.017 (0.019)	Loss 0.0925 (0.0808)	MAE 0.295 (0.345)
Epoch: [1][90/116]	Time 0.329 (0.329)	Data 0.017 (0.019)	Loss 0.0497 (0.0798)	MAE 0.272 (0.342)
Epoch: [1][100/116]	Time 0.308 (0.328)	Da

  7%|▋         | 2/30 [21:46<6:59:37, 899.19s/it]  

...

Test: [0/4]	Time 0.046 (0.046)	Loss 0.0180 (0.0180)	MAE 0.151 (0.151)
 * MAE 0.176
Epoch: [28][0/116]	Time 0.372 (0.372)	Data 0.051 (0.051)	Loss 0.0337 (0.0337)	MAE 0.165 (0.165)
Epoch: [28][10/116]	Time 0.291 (0.327)	Data 0.016 (0.021)	Loss 0.0201 (0.0407)	MAE 0.167 (0.233)
Epoch: [28][20/116]	Time 0.346 (0.327)	Data 0.018 (0.020)	Loss 0.0346 (0.0421)	MAE 0.209 (0.237)
Epoch: [28][30/116]	Time 0.315 (0.325)	Data 0.017 (0.019)	Loss 0.0287 (0.0352)	MAE 0.193 (0.216)
Epoch: [28][40/116]	Time 0.317 (0.325)	Data 0.017 (0.019)	Loss 0.0176 (0.0341)	MAE 0.133 (0.208)
Epoch: [28][50/116]	Time 0.330 (0.324)	Data 0.018 (0.019)	Loss 0.0185 (0.0323)	MAE 0.161 (0.201)
Epoch: [28][60/116]	Time 0.326 (0.324)	Data 0.018 (0.019)	Loss 0.0108 (0.0324)	MAE 0.123 (0.203)
Epoch: [28][70/116]	Time 0.323 (0.325)	Data 0.017 (0.019)	Loss 0.0355 (0.0319)	MAE 0.280 (0.203)
Epoch: [28][80/116]	Time 0.307 (0.325)	Data 0.017 (0.019)	Loss 0.0391 (0.0320)	MAE 0.203 (0.205)
Epoch: [28][90/116]	Time 0.323 (0.326)	Data 0

 97%|█████████▋| 29/30 [38:46<00:37, 37.88s/it]

Test: [0/4]	Time 0.047 (0.047)	Loss 0.1728 (0.1728)	MAE 0.189 (0.189)
 * MAE 0.150
Epoch: [29][0/116]	Time 0.383 (0.383)	Data 0.054 (0.054)	Loss 0.0222 (0.0222)	MAE 0.170 (0.170)
Epoch: [29][10/116]	Time 0.326 (0.324)	Data 0.018 (0.021)	Loss 0.0181 (0.0238)	MAE 0.160 (0.173)
Epoch: [29][20/116]	Time 0.315 (0.326)	Data 0.018 (0.020)	Loss 0.0440 (0.0277)	MAE 0.168 (0.172)
Epoch: [29][30/116]	Time 0.308 (0.329)	Data 0.018 (0.019)	Loss 0.0169 (0.0294)	MAE 0.155 (0.184)
Epoch: [29][40/116]	Time 0.313 (0.328)	Data 0.018 (0.019)	Loss 0.0161 (0.0279)	MAE 0.145 (0.180)
Epoch: [29][50/116]	Time 0.319 (0.328)	Data 0.018 (0.019)	Loss 0.0245 (0.0270)	MAE 0.169 (0.177)
Epoch: [29][60/116]	Time 0.309 (0.328)	Data 0.017 (0.019)	Loss 0.0146 (0.0253)	MAE 0.136 (0.173)
Epoch: [29][70/116]	Time 0.332 (0.327)	Data 0.018 (0.019)	Loss 0.0129 (0.0244)	MAE 0.126 (0.169)
Epoch: [29][80/116]	Time 0.299 (0.326)	Data 0.018 (0.019)	Loss 0.0177 (0.0236)	MAE 0.163 (0.166)
Epoch: [29][90/116]	Time 0.322 (0.326)	Data 0

100%|██████████| 30/30 [39:23<00:00, 78.80s/it]

Test: [0/4]	Time 0.049 (0.049)	Loss 0.1324 (0.1324)	MAE 0.225 (0.225)
 * MAE 0.207


In [59]:
ls # checkpoint.pth.tar generated

cgcnn/
checkpoint2.pth.tar
checkpoint_46744_energy_per_atom.pth.tar
checkpoint.pth.tar
data/
LICENSE
main.py
model_best2.pth.tar
model_best_46744_energy_per_atom.pth.tar
model_best.pth.tar
predict.py
pre-trained/
README.md
SrTiO3_folder/
test_results.csv


In [ ]:
# test best model
print('----------Evaluate Model on Test Set----------')
best_checkpoint = torch.load('model_best.pth.tar')
model.load_state_dict(best_checkpoint['state_dict'])

validate(val_loader = test_loader,
         model = model,
         criterion = criterion,
         normalizer = normalizer,
         test = True)

----------Evaluate Model on Test Set----------
Test: [0/4]	Time 13.123 (13.123)	Loss 0.0155 (0.0155)	MAE 0.110 (0.110)
 ** MAE 0.110


tensor(0.1101)

In [ ]:
cd SrTiO3_folder/

/content/drive/My Drive/cgcnn/SrTiO3_folder


In [ ]:
ls

atom_init.json             new_SrTiO3_Ti_delta_1.cif
new_SrTiO3_Ti_delta_0.cif  new_SrTiO3_Ti_delta_2.cif


## Graphene and Diamond

In [ ]:
import pymatgen
from pymatgen.ext.matproj import MPRester
from tqdm import tqdm

data = {}
with MPRester("pyvjZNMMGjtZskXhTQb") as m:
    
    for mp_id in ['mp-66', 'mp-48']:
        data[mp_id] = (m.query(criteria={'material_id': mp_id},
                               properties=['energy_per_atom', 'structure']))


In [ ]:
print(data['mp-48'])
print(data['mp-66'])

In [ ]:
# 1. new_id_prop_data 생성
new_id_prop_data = []
for key, value in data.items():
    if value == []:
        pass
    else:
        new_id_prop_data.append([key, value[0]['energy_per_atom']])

In [ ]:
len(new_id_prop_data)

In [ ]:
# new_crystal 정의
idx = 1
cif_id, target = new_id_prop_data[idx]
cif_id, target

In [ ]:
new_crystal = data[cif_id][0]['structure']
new_crystal

## Conclusion

* energy_per_atom을 target_property로 하여 CGCNN 가동
* 왜 안 돼!